In [1]:
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from config import *
from typing import Tuple

from data.util import split_weekdays_and_weekends

from prediction_models.dbn import *
from prediction_models.kelm import *

%reload_ext autoreload
%autoreload 2

In [2]:
DBN_HIDDEN_LAYER_SIZES = CONFIG['DBN_HIDDEN_LAYER_SIZES']
GIBBS_SAMPLING_STEPS = CONFIG['GIBBS_SAMPLING_STEPS']
READ_START_DATE = datetime.strptime(CONFIG['READ_START_DATE'], DATE_FORMAT)
READ_END_DATE = datetime.strptime(CONFIG['READ_END_DATE'], DATE_FORMAT)
TRAIN_START_DATE = datetime.strptime(CONFIG['TRAIN_START_DATE'], DATE_FORMAT)
TRAIN_END_DATE = datetime.strptime(CONFIG['TRAIN_END_DATE'], DATE_FORMAT)
TIME_WINDOW_LENGTH = CONFIG['TIME_WINDOW_LENGTH']

### Prepare training dataset

Load the matrix $C$ constructed from $Q$

In [3]:
mat_c = torch.load(out_path('mat_c.pt'), map_location=device)
TIME_DIM, SPACE_DIM = mat_c.shape
mat_c.shape

torch.Size([2880, 194])

Split $C$ into workdays and weekends data

In [4]:
mat_c_wd, mat_c_we = split_weekdays_and_weekends(mat_c, TRAIN_START_DATE, TRAIN_END_DATE)
assert mat_c_wd.shape[1] == mat_c_we.shape[1] == mat_c.shape[1]
assert mat_c_wd.shape[0] + mat_c_we.shape[0] == mat_c.shape[0]
mat_c_wd.shape, mat_c_we.shape

(torch.Size([2112, 194]), torch.Size([768, 194]))

Create a dataset for pre-training RBMs

In [5]:
class BinaryVectorDataset(Dataset):
    def __init__(self, n_bits: int):
        self.n_bits = n_bits
        self.format_str = f'{{0:0{self.n_bits}b}}'
    
    def __len__(self) -> int:
        return 200
    
    def __getitem__(self, index: int) \
            -> Tuple[torch.TensorType, torch.TensorType]:
        bin_str = self.format_str.format(index)
        return torch.tensor([float(c) for c in bin_str]), torch.tensor([index,])


class RandomBinaryVectorDataset(Dataset):
    def __init__(self, n_samples, bits_per_sample):
        self.n_samples = n_samples
        self.bits_per_sample = bits_per_sample

    def __len__(self) -> int:
        return self.n_samples

    def __getitem__(self, index: int) -> torch.TensorType:
        return (torch.rand(self.bits_per_sample) > 0.5).to(dtype=torch.float32)


dbn_pre_train_loader = DataLoader(RandomBinaryVectorDataset(10000, TIME_WINDOW_LENGTH), batch_size=256)

### Pre-train RBMs inside DBN

Train to reconstruct all possible combinations of 0's and 1's

In [6]:
dbn = DBN(TIME_WINDOW_LENGTH, DBN_HIDDEN_LAYER_SIZES, k=GIBBS_SAMPLING_STEPS).to(device)
pre_train_dbn(dbn, dbn_pre_train_loader, device, print_every=1)

Epoch 0:
	RBM 0: Loss=61.42926788330078
	RBM 1: Loss=39.32808303833008
	RBM 2: Loss=27.729629516601562
	RBM 0: Loss=57.6955680847168
	RBM 1: Loss=37.00298309326172
	RBM 2: Loss=23.515174865722656
	RBM 0: Loss=52.65970230102539
	RBM 1: Loss=34.04985809326172
	RBM 2: Loss=19.740337371826172
	RBM 0: Loss=50.025856018066406
	RBM 1: Loss=31.253135681152344
	RBM 2: Loss=16.24236297607422
	RBM 0: Loss=45.20793914794922
	RBM 1: Loss=27.484661102294922
	RBM 2: Loss=13.19510269165039
	RBM 0: Loss=42.40595245361328
	RBM 1: Loss=24.722759246826172
	RBM 2: Loss=10.831466674804688
	RBM 0: Loss=38.32177734375
	RBM 1: Loss=21.101181030273438
	RBM 2: Loss=9.525440216064453
	RBM 0: Loss=36.12704086303711
	RBM 1: Loss=19.007648468017578
	RBM 2: Loss=8.027523040771484
	RBM 0: Loss=33.1251335144043
	RBM 1: Loss=16.157119750976562
	RBM 2: Loss=7.11517333984375
	RBM 0: Loss=30.53848648071289
	RBM 1: Loss=13.24569320678711
	RBM 2: Loss=6.250339508056641
	RBM 0: Loss=29.156158447265625
	RBM 1: Loss=11.86526489

DBN(
  (rbms): ModuleList(
    (0-2): 3 x RBM()
  )
)

#### Train prediction of the first section

Define the loss function

In [7]:
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(dbn.parameters())
print([p.shape for p in dbn.parameters()])

[torch.Size([1, 60]), torch.Size([1, 32]), torch.Size([32, 60]), torch.Size([1, 32]), torch.Size([1, 32]), torch.Size([32, 32]), torch.Size([1, 32]), torch.Size([1, 32]), torch.Size([32, 32])]


In [11]:
kelm = KELM()
kelm.random_fit(device, SPACE_DIM, DBN_HIDDEN_LAYER_SIZES[-1], 1)
dbn_features = []
dbn_labels = []
dbn.train()
for start_time in range(TIME_DIM - TIME_WINDOW_LENGTH - 1):
    end_time = start_time + TIME_WINDOW_LENGTH
    train_window = mat_c[start_time:end_time].T
    labels = mat_c[end_time]
    features = dbn(train_window)
    
    kelm_X = torch.concat(dbn_features + [features,], dim=0)
    kelm_y = torch.concat(dbn_labels + [labels,])[:, None]

    optim.zero_grad()
    pred = kelm(kelm_X)
    loss = loss_fn(pred, kelm_y)
    loss.backward()
    optim.step()

    print(loss.item())

    dbn_features.append(features.detach())
    dbn_labels.append(labels.detach())

    kelm_X = torch.concat(dbn_features, dim=0)
    kelm_y = torch.concat(dbn_labels)[:, None]
    kelm.fit(kelm_X, kelm_y)

    # kelm.fit(features, label_window)
    # predictions = kelm(features) # [60, 60]
    # print(predictions.shape)
    # print(loss_fn(predictions, label_window))
    # USE THIS TO TRAIN DBN
dbn.train(False)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!